In [1]:
import pandas as pd
import jieba
import numpy

df_news = pd.read_table('/Users/eamonyin/Downloads/迪哥nlp/练习/贝叶斯-新闻分类/data/val.txt',names=['category','theme','URL','content'],encoding='utf-8')
df_news = df_news.dropna()

content = df_news.content.values.tolist()
content[1000]

'阿里巴巴集团昨日宣布，将在集团管理层面设立首席数据官岗位（Ｃｈｉｅｆ\u3000Ｄａｔａ\u3000Ｏｆｆｉｃｅｒ），阿里巴巴Ｂ２Ｂ公司ＣＥＯ陆兆禧将会出任上述职务，向集团ＣＥＯ马云直接汇报。＞菹ぃ和６月初的首席风险官职务任命相同，首席数据官亦为阿里巴巴集团在完成与雅虎股权谈判，推进“ｏｎｅ\u3000ｃｏｍｐａｎｙ”目标后，在集团决策层面新增的管理岗位。０⒗锛团昨日表示，“变成一家真正意义上的数据公司”已是战略共识。记者刘夏'

In [14]:
content_S = []
for line in content:
    current_segment = jieba.lcut(line)
    if len(current_segment) > 1 and current_segment != '\r\n': #换行符
        content_S.append(current_segment)

df_content=pd.DataFrame({'content_S':content_S}) 

stopwords=pd.read_csv("/Users/eamonyin/Downloads/迪哥nlp/练习/贝叶斯-新闻分类/stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')

def drop_stopwords(contents,stopwords):
    contents_clean = []
    all_words = [] #做词云，做准备
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word)) #做词云，做准备
        contents_clean.append(line_clean)
    return contents_clean,all_words
    #print (contents_clean)
        

contents = df_content.content_S.values.tolist()    
stopwords = stopwords.stopword.values.tolist()
contents_clean,all_words = drop_stopwords(contents,stopwords)

df_all_words=pd.DataFrame({'all_words':all_words})
df_all_words.head()

import numpy as np

words_count = df_all_words.groupby(by=['all_words'])['all_words'].agg(count=np.size)
words_count.head()

# print(contents_clean[10])

,count
all_words,
D,4
F,3
P,3
S,2
T,3


In [3]:
import jieba.analyse
index = 1000
print (df_news['content'][index])
content_S_str = "".join(content_S[index])  
print ("  ".join(jieba.analyse.extract_tags(content_S_str, topK=5, withWeight=False)))

阿里巴巴集团昨日宣布，将在集团管理层面设立首席数据官岗位（Ｃｈｉｅｆ　Ｄａｔａ　Ｏｆｆｉｃｅｒ），阿里巴巴Ｂ２Ｂ公司ＣＥＯ陆兆禧将会出任上述职务，向集团ＣＥＯ马云直接汇报。＞菹ぃ和６月初的首席风险官职务任命相同，首席数据官亦为阿里巴巴集团在完成与雅虎股权谈判，推进“ｏｎｅ　ｃｏｍｐａｎｙ”目标后，在集团决策层面新增的管理岗位。０⒗锛团昨日表示，“变成一家真正意义上的数据公司”已是战略共识。记者刘夏
阿里巴巴  集团  首席  岗位  数据


In [13]:
from gensim import corpora,models,similarities
import gensim

#做映射，词袋
dictionary = corpora.Dictionary(contents_clean)
corpus = [dictionary.doc2bow(sentence) for sentence in contents_clean]

lda = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary, num_topics=20) #类似Kmeans自己指定K值

#一号分类结果
print(lda.print_topic(1, topn=5))

0.005*"赛" + 0.003*"阴道炎" + 0.003*"文物" + 0.002*"中" + 0.002*"Ｉ"


In [15]:
for topic in lda.print_topics(num_topics=20, num_words=5):
    print (topic)

(0, '0.007*"中" + 0.007*"说" + 0.005*"比赛" + 0.004*"球队" + 0.004*"Ｔ"')
(1, '0.005*"赛" + 0.003*"阴道炎" + 0.003*"文物" + 0.002*"中" + 0.002*"Ｉ"')
(2, '0.007*"中" + 0.006*"发展" + 0.005*"学校" + 0.004*"工作" + 0.004*"中国"')
(3, '0.008*"节目" + 0.008*"中" + 0.006*"中国" + 0.005*"粉丝" + 0.004*"主持人"')
(4, '0.013*"中国" + 0.007*"中" + 0.005*"Ｖ" + 0.003*"Ｓ" + 0.003*"Ｍ"')
(5, '0.009*"欧洲杯" + 0.005*"中" + 0.005*"球迷" + 0.003*"说" + 0.003*"前锋"')
(6, '0.005*"中" + 0.005*"性爱" + 0.003*"球队" + 0.003*"意大利" + 0.003*"查查"')
(7, '0.018*"男人" + 0.014*"女人" + 0.008*"说" + 0.007*"中" + 0.007*"孩子"')
(8, '0.010*"该剧" + 0.009*"万" + 0.008*"号" + 0.004*"中" + 0.004*"转"')
(9, '0.018*"饰演" + 0.006*"中" + 0.004*"说" + 0.003*"Ｉ" + 0.003*"高考"')
(10, '0.013*"电影" + 0.011*"导演" + 0.011*"中" + 0.007*"影片" + 0.005*"演员"')
(11, '0.008*"学生" + 0.005*"学校" + 0.004*"孩子" + 0.003*"小学" + 0.003*"中"')
(12, '0.008*"肌肤" + 0.007*"皮肤" + 0.006*"中" + 0.004*"说" + 0.004*"乳房"')
(13, '0.006*"中" + 0.004*"说" + 0.004*"考生" + 0.003*"官兵" + 0.003*"Ｐ"')
(14, '0.028*"ｅ" + 0.028*"ａ" + 0.022*"ｏ" + 0

In [16]:
#转化结构
df_train=pd.DataFrame({'contents_clean':contents_clean,'label':df_news['category']})

#创建label映射
label_mapping = {"汽车": 1, "财经": 2, "科技": 3, "健康": 4, "体育":5, "教育": 6,"文化": 7,"军事": 8,"娱乐": 9,"时尚": 0}
df_train['label'] = df_train['label'].map(label_mapping)
df_train.head()


,contents_clean,label
0,"[经销商, 电话, 试驾, 订车, Ｕ, 憬, 杭州, 滨江区, 江陵, 路, 号, 转, ...",1
1,"[呼叫, 热线, 服务, 邮箱, ｋ, ｆ, ｐ, ｅ, ｏ, ｐ, ｌ, ｅ, ｄ, ａ,...",1
2,"[Ｍ, Ｉ, Ｎ, Ｉ, 品牌, 二月, 公布, 最新, Ｍ, Ｉ, Ｎ, Ｉ, 新, 概念...",1
3,"[清仓, 甩卖, 一汽, 夏利, Ｎ, 威志, Ｖ, 低至, 万, 启新, 中国, 一汽, ...",1
4,"[日内瓦, 车展, 见到, 高尔夫, 家族, 新, 成员, 高尔夫, 敞篷版, 款, 全新,...",1


In [32]:
from sklearn.model_selection import train_test_split
# 语料分成测试集和训练集
x_train, x_test, y_train, y_test = train_test_split(df_train['contents_clean'].values, df_train['label'].values, random_state=1)

x_train[0][1]

'上海'

In [41]:
words = []
for line_index in range(len(x_train)):
    try:
        #x_train[line_index][word_index] = str(x_train[line_index][word_index])
        words.append(' '.join(x_train[line_index]))
    except:
        print (line_index)
# print(words[0])   
# 
# print('---------')

from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(analyzer='word', max_features=4000,  lowercase = False)
vec.fit(words)

# print(words[0]) 
# 
print('-----向量处理----')
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
print(vec.get_feature_names_out())
print(vec.transform(words).toarray())
# print('---------')
# classifier.fit(vec.transform(words), y_train)

# from sklearn.naive_bayes import MultinomialNB
# classifier = MultinomialNB()
# classifier.fit(vec.transform(words), y_train)
# 评估分类器 classifier 在训练数据上的性能表现。
classifier.score(vec.transform(words), y_train)



-----向量处理----
['一下子' '一个月' '一举' ... '默契' '鼓励' '齐全']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


NotFittedError: This MultinomialNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [30]:
classifier.fit(vec.transform(words), y_train)

MultinomialNB()